In [1]:
import pandas as pd
from pathlib import Path
import os
from collections import defaultdict
from pythainlp.tag.blackboard import post_process

In [2]:
train = Path(os.path.join('.','blackboard-treebank','thai10'))
train_files = [f for f in train.iterdir() if not f.name.startswith('.')]
def readfile(path):
    _data = []
    with open(path,'r',encoding="utf-8-sig") as f:
        _temp = [i for i in f.read().split("\n\n") if i != '']
    for j in _temp:
        _temp_data = []
        bad_data = False
        for i in j.split("\n"):
            if i.startswith('#') == False and not bad_data:
                _t = i.strip().split("\t")
                if _t[1]!="$$":
                    if _t[3]=="N":
                        _temp_data=[]
                        bad_data=True
                    else:
                        _temp_data.append((_t[1],_t[3]))
        if _temp_data!=[]:
            _data.append(_temp_data)
    return _data
train_data = []
for file in train_files:
    train_data.extend(readfile(file))
train_data_ud = [post_process(i,to_ud=True) for i in train_data]
pos_dict = defaultdict(list)
for i in train_data_ud:
    _word= [k for k,_ in i]
    _pos = ' / '.join([k for _,k in i])
    pos_dict[_pos].append(_word)
temp="จงสร้างประโยคตามโครงสร้าง {pos}:"
inputs=[]
targets=[]
for i in sorted(list(pos_dict.keys())):
    if i.count("/")<1:
        continue
    for k in pos_dict[i]:
        inputs.append(temp.format(pos=i))
        targets.append(' '.join(k))
df_train = {
    "inputs":inputs,
    "targets":targets,
}

In [3]:
path = "th_pud-ud-test.conllu"
test_data_ud = readfile(path)
pos_dict = defaultdict(list)
for i in test_data_ud:
    _word= [k for k,_ in i]
    _pos = ' / '.join([k for _,k in i])
    pos_dict[_pos].append(_word)
inputs=[]
targets=[]
for i in sorted(list(pos_dict.keys())):
    if i.count("/")<1:
        continue
    for k in pos_dict[i]:
        inputs.append(temp.format(pos=i))
        targets.append(' '.join(k))
df_test = {
    "inputs":inputs,
    "targets":targets,
}

In [4]:
from datasets import Dataset, DatasetDict

In [5]:
ds = DatasetDict()
ds['test'] = Dataset.from_dict(df_test)
ds['train'] = Dataset.from_dict(df_train)

In [8]:
ds

DatasetDict({
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 1000
    })
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 130454
    })
})

In [9]:
ds["train"].to_pandas()

,inputs,targets
0,จงสร้างประโยคตามโครงสร้าง ADJ / ADJ / AUX / VE...,ก่อน อื่น ต้อง ขอ แสดง ความ ยินดี กับ ยานินา ด้วย
1,จงสร้างประโยคตามโครงสร้าง ADJ / ADJ / NOUN / A...,ตลอด ทั้ง วัน ของ วันที่ 17 ส.ค. มี กระแส ข่าว...
2,จงสร้างประโยคตามโครงสร้าง ADJ / ADJ / NOUN / N...,อีก สัก พัก หลักฐาน ต่าง จะ ออก มา
3,จงสร้างประโยคตามโครงสร้าง ADJ / ADJ / NUM / NO...,อีก ประมาณ 2-3 วัน จะ ให้การ กับ เจ้าหน้าที่ ได้
4,จงสร้างประโยคตามโครงสร้าง ADJ / ADJ / NUM / NU...,อีก ประมาณ เก้า พัน สาม ร้อย ล้าน บาท แล้ว ก็ ...
...,...,...
130449,จงสร้างประโยคตามโครงสร้าง VERB / VERB / VERB /...,ค้าน ประกัน หวั่น หลบหนี เจ้าตัว ยัง ให้การ ปฏ...
130450,จงสร้างประโยคตามโครงสร้าง VERB / VERB / VERB /...,ขาย ดี มาก มี เท่า ไร ก็ ขาย หมด ขณะนี้ โรง งาน
130451,จงสร้างประโยคตามโครงสร้าง VERB / VERB / VERB /...,ชวน เตือน อย่า มอง ข้าม นปช.
130452,จงสร้างประโยคตามโครงสร้าง VERB / VERB / VERB /...,เน้น ลุย กวาดล้าง จริงจัง ตาม กฎเหล็ก


In [10]:
ds["test"].to_pandas()

,inputs,targets
0,จงสร้างประโยคตามโครงสร้าง ADJ / ADP / NOUN / A...,อุดม ด้วย แร่ธาตุ โดย เป็น แหล่ง ของ ทองคำ เพช...
1,จงสร้างประโยคตามโครงสร้าง ADJ / ADP / NOUN / D...,การเสมอ ใน เกม ที่ คุณ เหนือ กว่า เป็น เรื่อง ...
2,จงสร้างประโยคตามโครงสร้าง ADJ / ADP / PRON / A...,"แน่นอน ว่า นี่ คือ การเลือกตั้ง ไม่ ใช่ "" มหาศ..."
3,จงสร้างประโยคตามโครงสร้าง ADJ / ADP / PROPN / ...,แน่นอน ว่า จีน จะ จัด แสดง ยุทโธปกรณ์ อื่น ๆ อ...
4,จงสร้างประโยคตามโครงสร้าง ADJ / ADP / VERB / N...,โชคดี ที่ มี คน จาก โซนี่ ออสเตรเลีย ที่ มา ปร...
...,...,...
995,จงสร้างประโยคตามโครงสร้าง VERB / VERB / PART /...,การค้า พัฒนา ขึ้น จาก การตั้ง ด่าน ชั้น นอก เพ...
996,จงสร้างประโยคตามโครงสร้าง VERB / VERB / PART /...,กระโดด ข้าม ไป ปี 2016 แล้ว เรื่อง นี้ จะ ยิ่ง...
997,จงสร้างประโยคตามโครงสร้าง VERB / VERB / PROPN ...,การจัด จำแนก เบอร์เกรอน เป็น การจัด ประเภท มวล...
998,จงสร้างประโยคตามโครงสร้าง VERB / VERB / VERB /...,ก่อ ให้ เกิด เส้น ทาง การค้า การล่า อาณานิคม แ...


In [11]:
ds.push_to_hub("pythainlp/Thai-Pos-prompt",private=False)

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]